In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Definizione di una CNN classica
class ClassicCNN(nn.Module):
    def __init__(self):
        super(ClassicCNN, self).__init__()
        # Layer convoluzionali
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)  # Convoluzione: 1 canale in ingresso, 32 canali in uscita
        self.fc1 = nn.Linear(8 * 28 * 28, 128)  # Fully connected: 64*7*7 è il numero di neuroni dopo il pooling
        self.fc2 = nn.Linear(128, 10)  # Ultimo layer per le 10 classi di MNIST

    def forward(self, x):
        # Passaggio attraverso i layer convoluzionali
        x = torch.relu(self.conv1(x)) # ReLU e max pooling dopo la convoluzione
        
        # Flatten per passare ai layer fully connected
        x = x.view(x.size(0), -1)  # Flatten l'output dei layer convoluzionali

        # Passaggio attraverso i layer fully connected
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # Output finale (logits per 10 classi)
        return x

# Caricamento dataset MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])  # Normalizzazione
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassicCNN().to(device)  # Creazione del modello e trasferimento su GPU/CPU
criterion = nn.CrossEntropyLoss()  # Funzione di perdita
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Ottimizzatore Adam

# Addestramento della rete
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

# Valutazione del modello sui dati di test
model.eval()  # Imposta il modello in modalità di valutazione
correct = 0
total = 0
with torch.no_grad():  # Disabilita il calcolo dei gradienti per il test
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')

Epoca [1/5], Loss: 0.2395, Accuracy: 92.92%
Epoca [2/5], Loss: 0.0819, Accuracy: 97.53%
Epoca [3/5], Loss: 0.0537, Accuracy: 98.39%
Epoca [4/5], Loss: 0.0366, Accuracy: 98.83%
Epoca [5/5], Loss: 0.0267, Accuracy: 99.17%
Accuracy sui test: 98.22%


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms

# Funzione per passare da dominio del tempo a dominio della frequenza
def to_frequency_domain(x):
    return torch.fft.fft2(x)

# Funzione per tornare al dominio del tempo
def to_time_domain(x):
    return torch.fft.ifft2(x).real

# Layer di moltiplicazione elemento per elemento nel dominio della frequenza
class FrequencyConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(FrequencyConv, self).__init__()
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels
        
        # Definizione dei pesi nel dominio della frequenza
        self.weights = nn.Parameter(torch.randn(1, out_channels, in_channels, kernel_size, kernel_size, dtype=torch.cfloat))

    def forward(self, x):
        batch_size, _, H, W = x.shape
        x_freq = to_frequency_domain(x)  # Trasformata di Fourier
        
        # Creazione di un kernel della stessa dimensione delle feature in ingresso
        kernel_padded = torch.zeros((batch_size, self.out_channels, self.in_channels, H, W), dtype=torch.cfloat, device=x.device)
        kernel_padded[:, :, :, :self.kernel_size, :self.kernel_size] = self.weights  # Padding
        
        kernel_freq = to_frequency_domain(kernel_padded)  # Trasformata di Fourier del kernel
        
        out_freq = x_freq.unsqueeze(1) * kernel_freq  # Moltiplicazione elemento per elemento
        out_time = to_time_domain(out_freq)  # Ritorno nel dominio del tempo
        
        return out_time.sum(dim=2)  # Somma lungo i canali in ingresso
    
# Definizione di CEMNet
class CEMNet(nn.Module):
    def __init__(self):
        super(CEMNet, self).__init__()
        self.conv1 = FrequencyConv(1, 8, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

        
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Caricamento dataset MNIST
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CEMNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Addestramento della rete
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

# Valutazione del modello sui dati di test
model.eval()  # Imposta il modello in modalità di valutazione
correct = 0
total = 0
with torch.no_grad():  # Disabilita il calcolo dei gradienti per il test
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')

Epoca [1/5], Loss: 0.4056, Accuracy: 88.72%
Epoca [2/5], Loss: 0.3193, Accuracy: 91.00%
Epoca [3/5], Loss: 0.3016, Accuracy: 91.35%
Epoca [4/5], Loss: 0.2906, Accuracy: 91.69%
Epoca [5/5], Loss: 0.2836, Accuracy: 91.99%
Accuracy sui test: 91.74%


ihodlkjsaldfnlqwesajn